In [1]:
import pandas as pd
import numpy as np

from patsy import dmatrices
from sklearn import preprocessing

from sklearn import linear_model
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('/Users/jdmendoza/Desktop/phone_price_tool/training/dataset/a7bad4b1-20c1-4049-88c0-a9db77bb56e4.csv')


In [3]:
# Most recent data point
max(pd.to_datetime(data['date'].dropna()))

Timestamp('2020-02-08 00:00:00')

In [4]:
# Excluding some data for now like sold date
subset = ['color', 'condition', 'price', 'storage', 'carrier', 'model']
data = data.dropna().drop_duplicates().reset_index(drop=True)[subset]

In [5]:
# Auto make dummy variables
y, X = dmatrices("price ~ color + storage + condition + carrier + model", data, return_type='dataframe')


In [6]:
scaler_1 = preprocessing.MinMaxScaler(feature_range=(0,1))
scaler_2 = preprocessing.MinMaxScaler(feature_range=(0,1))
X_scaled = scaler_1.fit_transform(X)
y_scaled = scaler_2.fit_transform(y)

In [7]:
model = linear_model.LinearRegression()
scores = cross_val_score(model, X_scaled, y_scaled, cv=5, scoring='neg_mean_squared_error')
print ("Cross validation MSE scores: {} (+/- {})".format(-scores.mean(), scores.std() * 2))

Cross validation MSE scores: 0.0015850291724314725 (+/- 0.00041091398592251973)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [8]:
model.fit(X_scaled, y_scaled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [9]:
def vectorize(feature, val):
    columns = [x for x in X.columns if feature in x]
    return [1 if (val.lower() in y.lower()) else 0 for y in columns]

In [10]:
my_phone = {'color':'white', 'condition':'good', 'storage':128, 'carrier':'att', 'model':'apple-iphone-8-plus'}

In [11]:
my_phone_vec = list(model.intercept_) + vectorize('color', my_phone['color']) + vectorize('condition',my_phone['condition'])\
+ vectorize('carrier',my_phone['carrier']) + vectorize('model',my_phone['model']) + [my_phone['storage']]



In [12]:
# Quick Test
scaler_2.inverse_transform(model.predict(scaler_1.transform([my_phone_vec]))).ravel()[0]

320.79711670420204